In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.regression.rolling import RollingOLS
from functools import partial
import sys
import math
pd.options.display.float_format = "{:,.4f}".format
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings("ignore")
import os
from scipy.stats import norm

In [ ]:
#import data
df = pd.read_excel('/Users/yiningqu/Desktop/dfa_analysis_data.xlsx', sheet_name = 'factors').set_index('Date')


#import data
raw_data = pd.read_excel('../data/momentum_data.xlsx',sheet_name = None)

#import data with multiple sheets
sheets = list(raw_data.keys())
ff_factors = raw_data[sheets[1]].set_index('Date')
momentum = raw_data[sheets[2]].set_index('Date')
mom_deciles = raw_data[sheets[3]].set_index('Date')
tercile_port = raw_data[sheets[4]].set_index('Date')
rf = raw_data[sheets[5]].set_index('Date')
ff_factors['UMD'] = momentum['UMD']


#import data with skipped rows
ltcm = pd.read_excel('../data/ltcm_exhibits_data.xlsx', sheet_name='Exhibit 2', skiprows=2, index_col=0, parse_dates=[0]).iloc[:-4, :]
ltcm.index.name = 'Date'

#import data with no 'Date'
spy = pd.read_excel('../data/gmo_analysis_data.xlsx', sheet_name='returns (total)', index_col=0, parse_dates=[0])[['SPY']]
us3m = pd.read_excel('../data/gmo_analysis_data.xlsx', sheet_name='risk-free rate', index_col=0, parse_dates=[0])[['US3M']]


#datetime: format date
ltcm.index = pd.to_datetime(ltcm.index, format='%Y-%m-%d') + pd.tseries.offsets.MonthEnd(0)


#drop data
factors = df.drop(['RF'], axis=1)


#subtract
portfolio_ret = portfolio.sub(risk_free.values).loc['1981':]
df.loc[:, ['LTCM Net', 'LTCM Gross', 'SPY']] = df.loc[:, ['LTCM Net', 'LTCM Gross', 'SPY']].subtract(df['US3M'], axis=0)

#rename
ts = ts.rename(columns={'Market beta':'MKT','Value beta':'HML','Momentum beta':'UMD'})

#log 
#log(1+x)
for col in risk_free_rates.columns:
    risk_free_rates[col] = risk_free_rates[col]
    risk_free_rates['log_'+col] = np.log(1+risk_free_rates[col])
#log(x)
for col in fx_rates.columns:
    fx_rates['log_'+ col] = fx_rates[col].apply(lambda x: np.log(x))

for col in fx_rates.columns:
    fx_rates['log_'+col] = np.log(fx_rates[col])

#合并combine
strat_var= pd.concat([strat_summary_df.loc[:,['VaR (0.05)']],
                      market_summary.loc[:,['VaR (0.05)']],
                      gmo_summary.loc[:,['VaR (0.05)']]])

oos_r2_sum = pd.concat([OOS_r2_dp,OOS_r2_ep,OOS_r2_epdp,OOS_r2_all])
    
#join
#join three dataset
df = ltcm.join(spy, how='inner').join(us3m, how='inner') 

#join two
pre_case = ta.calc_performance_metrics(factors.loc[:'2014']).iloc[:, :3]
post_case = ta.calc_performance_metrics(factors.loc['2015':]).iloc[:, :3]
pre_post = pre_case.join(post_case, lsuffix=' Pre', rsuffix=' Post')


#generate call & put option
#np.where(condition, true, false)
spy_er_ltcm['SPY_call'] = np.where(spy_er_ltcm['SPY'] > 0.03, spy_er_ltcm['SPY'] - 0.03, 0)
spy_er_ltcm['SPY_put'] = np.where(-0.03 - spy_er_ltcm['SPY'] >= 0, -0.03 - spy_er_ltcm['SPY'], 0)

#display max & min
print("The asset with the best Sharpe ratio is: ")
display(summary_stats_excess_returns[summary_stats_excess_returns['Sharpe'] == summary_stats_excess_returns['Sharpe'].max()][['Sharpe']])
print("The asset with the worst Sharpe ratio is: ")
display(summary_stats_excess_returns[summary_stats_excess_returns['Sharpe'] == summary_stats_excess_returns['Sharpe'].min()][['Sharpe']])

#find min & max corr 
correlation_matrix = excess_returns.corr()
corr_rank = correlation_matrix.unstack().sort_values().to_frame('Correlations')
corr_rank = corr_rank[corr_rank['Correlations']!=1]
pair_max = corr_rank.index[-1]
pair_min = corr_rank.index[0]


#calculate frequency (excess r < r_var(0.05))
(df['Excess SPY Returns'] < Historic_VaR).sum()/len(Historic_VaR.dropna())*100:.2f

#calculate rolling frequency 
var_temp = VaR.dropna()
frequency =  var_temp.apply(lambda x: rets.loc[var_temp.index] < x).mean().to_frame('frequency')
frequency.style.format('{:,.2%}')

#correlation
summary['Corr Mkt'] = np.corrcoef(np.array(sub_factors['MKT']), np.array(sub_mom[port]))[0,1]

#construct dataframe:
dp_forecast = pd.DataFrame([[beta_dp,alpha,r_squared]],columns=['DP Beta','Alpha','R-Squared'],index = ['DP'])
stats_list = pd.DataFrame(columns=['Mean', 'Volatility', 'Sharpe_ratio'], index=hedge_fund.columns)

#St+1 - St
strat = fx_rates[v].diff()

#merge
strat_rtn = strat_rtn.merge(forecasting[['GLD']], left_index = True, right_index = True)

# Performance_summary

In [ ]:
def performance_summary(return_data, period = 12):
    """ 
        Returns the Performance Stats for given set of returns
        Inputs: 
            return_data - DataFrame with Date index and Monthly Returns for different assets/strategies.
        Output:
            summary_stats - DataFrame with annualized mean return, vol, sharpe ratio. Skewness, Excess Kurtosis, Var (0.5) and
                            CVaR (0.5) and drawdown based on monthly returns. 
    """
    summary_stats = return_data.mean().to_frame('Mean').apply(lambda x: x*period)
    summary_stats['Volatility'] = return_data.std().apply(lambda x: x*np.sqrt(period))
    summary_stats['Sharpe Ratio'] = summary_stats['Mean']/summary_stats['Volatility']
    summary_stats['Skewness'] = return_data.skew()
    summary_stats['Excess Kurtosis'] = return_data.kurtosis()
    summary_stats['VaR (0.05)'] = return_data.quantile(.05, axis = 0)
    summary_stats['CVaR (0.05)'] = return_data[return_data <= return_data.quantile(.05, axis = 0)].mean()
    summary_stats['Min'] = return_data.min()
    summary_stats['Max'] = return_data.max()
    
    wealth_index = 1000*(1+return_data).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks

    summary_stats['Max Drawdown'] = drawdowns.min()
    summary_stats['Peak'] = [previous_peaks[col][:drawdowns[col].idxmin()].idxmax() for col in previous_peaks.columns]
    summary_stats['Bottom'] = drawdowns.idxmin()
    
    recovery_date = []
    for col in wealth_index.columns:
        prev_max = previous_peaks[col][:drawdowns[col].idxmin()].max()
        recovery_wealth = pd.DataFrame([wealth_index[col][drawdowns[col].idxmin():]]).T
        recovery_date.append(recovery_wealth[recovery_wealth[col] >= prev_max].index.min())
    summary_stats['Recovery'] = recovery_date
    
    return summary_stats

### two or more sample
#### Method 1:

In [ ]:
sub_1980 = factors.loc[:'1980']
sub_2001 = factors.loc['1981':'2001']
sub_2023 = factors.loc['2002':]

df_dict={'1926-1980' : sub_1980,
         '1981-2001' : sub_2001,
         '2002-2023' : sub_2023}

summary_lst = []
for key in df_dict.keys():
    summary_stats = performance_summary(df_dict[key], period = 12).loc[:,['Mean','Volatility','Sharpe Ratio','VaR (0.05)']]
    summary_stats['Period'] = key
    summary_stats= summary_stats.reset_index().rename(columns = {'index':'Factor'}).set_index(['Period','Factor'])
    summary_lst.append(summary_stats)

factor_summary = pd.concat(summary_lst)
factor_summary

In [ ]:
#将exchange rate和risk free rate两个dataset的column name 合并
fx_spot_map = {'log_GBP1M':'log_USUK'
                ,'log_EUR1M':'log_USEU'
                ,'log_CHF1M':'log_USSZ'
                ,'log_JPY1M':'log_USJP'
}

fx_hldg_lst = []
for k,v in fx_spot_map.items():
    #fx_rate shift(1) = St+1; fx_rate = St; risk_free shift(1) = Rt+1
    fx_hldg_excess_ret = fx_rates[v] - fx_rates[v].shift(1) + risk_free_rates[k].shift(1) - risk_free_rates['log_USD1M'].shift(1)
    fx_hldg_summary = ta.performance_summary(fx_hldg_excess_ret.to_frame().dropna())
    #change log_GBP1M to GBP
    fx_hldg_summary.index = [k[4:7]]
    fx_hldg_summary.index.name = 'Currency Held'
    fx_hldg_lst.append(fx_hldg_summary)

fx_hldg_perf_summary = pd.concat(fx_hldg_lst)
fx_hldg_perf_summary.loc[:, ['Mean','Volatility','Sharpe Ratio', 'Min', 'Max']]

# Tangency Weights

In [ ]:
def tangency_weights(returns, cov_mat = 1):
    
    if cov_mat ==1:
        cov_inv = np.linalg.inv((returns.cov()*12))
    else:
        cov = returns.cov()
        covmat_diag = np.diag(np.diag((cov)))
        covmat = cov_mat * cov + (1-cov_mat) * covmat_diag
        cov_inv = np.linalg.inv((covmat*12))  
        
    ones = np.ones(returns.columns[1:].shape) 
    mu = returns.mean()*12
    scaling = 1/(np.transpose(ones) @ cov_inv @ mu)
    tangent_return = scaling*(cov_inv @ mu) 
    tangency_wts = pd.DataFrame(index = returns.columns[1:], data = tangent_return, columns = ['Tangent Weights'] )
        
    return tangency_wts

* Compute the mean, volatility, and Sharpe ratio for the tangency portfolio corresponding to $\wtan$.

In [ ]:
# tangency weight * excess return
w_tan_summary = performance_summary(excess_returns @ w_t , annual_factor = 12)

In [ ]:
target_mean = .015
excess_returns = multi_asset_etf_excess_ret[multi_asset_etf_excess_ret.columns[1:]] 
w_t_without_tips.loc['TIP'] = 0
wts = pd.DataFrame(index = excess_returns.columns, columns = ['tangency','tangency with TIPS dropped',
                                                              'tangency with TIPS adjusted','equal weights',
                                                              'risk parity','regularized'])


wts.loc[:,'tangency'] = tangency_weights(multi_asset_etf_excess_ret, cov_mat = 1).values
wts.loc[:,'tangency with TIPS dropped'] = w_t_without_tips.values
wts.loc[:,'tangency with TIPS adjusted'] = w_t_adj_tips.values
wts.loc[:,'equal weights'] = 1/len(excess_returns)
wts.loc[:,'risk parity'] = 1/excess_returns.var()
#*********************************
wts.loc[:,'regularized'] = tangency_weights(multi_asset_etf_excess_ret, cov_mat = 0.5).values

#*********************************
wts *= target_mean / (excess_returns.mean()@wts)

wts

# regression_based_performance

In [ ]:
#one factor
def regression_based_performance(fund_ret,factor, rf,constant = True):
    """ 
        Returns the Regression based performance Stats for given set of returns and factors
        Inputs:
            factor - Dataframe containing monthly returns of the regressors
            fund_ret - Dataframe containing monthly excess returns of the regressand fund
            rf - Monthly risk free rate of return
        Output:
            summary_stats - (Beta of regression, treynor ratio, information ratio, alpha). 
    """
    if constant:
        X = sm.tools.add_constant(factor)
    else:
        X = factor
    y=fund_ret
    model = sm.OLS(y,X,missing='drop').fit()
    
    if constant:
        beta = model.params[1:]
        alpha = round(float(model.params['const']),6) *12

        
    else:
        beta = model.params
    treynor_ratio = ((fund_ret - rf).mean()*12)/beta[0]
    tracking_error = (model.resid.std()*np.sqrt(12))
    if constant:        
        information_ratio = model.params[0]*12/tracking_error
    r_squared = model.rsquared
    if constant:
        return (beta,treynor_ratio,information_ratio,alpha,r_squared,tracking_error,model.resid)
    else:
        return (beta,treynor_ratio,r_squared,tracking_error,model.resid)

In [ ]:
#one fund one factor
betas, treynor, alpha, r_squared, tracking_er_vol = regression_based_performance(
    merrill, hfri, risk_free, multiple=True, alpha=False)
coefficients_new = pd.DataFrame(betas, columns=['beta'], index=merrill.columns)

## Multiple sample

In [ ]:
#multiple sample
reg_sub_sample = []
for k,v in sub_samples.items():    
    y = gmo.loc[sub_samples[k][0]:sub_samples[k][1],['GMWAX']].dropna()
    X = gmo.loc[y.index[0]:y.index[-1],['SPY']]
    reg = regression_based_performance(X,y)
    beta_mkt = reg[0][0]
    alpha = reg[3]
    r_squared = reg[4]
    reg_sub_sample.append(pd.DataFrame([[beta_mkt,alpha,r_squared]],columns=['SPY Beta','Alpha','R-Squared'],index = [k]))

reg_performance = pd.concat(reg_sub_sample)
reg_performance

## Multiple factors

### method 1

In [ ]:
#multiple fund one factor
regression_stats_df = pd.DataFrame(columns=[
                                   'Market_Beta', 'Treynor_ratio', 
                                    'Information_ratio', 'Alpha', 'R_Squared'], index=hedge_fund.columns)
risk_free = merrill['USGG3M Index']
stats_list = pd.DataFrame(
    columns=['Mean', 'Volatility', 'Sharpe_ratio'], index=hedge_fund.columns)

for fund in hedge_fund:
    beta, treynor, information, alpha, r_squared, _ = regression_based_performance(
        hedge_fund[fund],merrill['SPY US Equity'], risk_free)
    regression_stats_df.loc[fund] = [
        beta[0], treynor, information, alpha, r_squared]

    cur_stats = stats(hedge_fund[fund].to_frame())
    stats_list.loc[fund] = [cur_stats['Mean'][0],
                            cur_stats['Volatility'][0], cur_stats['Sharpe_ratio'][0]]

### method 2

In [ ]:
#y=change in fx, x=interest rate difference --> whether follows UIP
fx_hldg_reg = []
for k,v in fx_spot_map.items():
    factor = risk_free_rates['log_USD1M'] - risk_free_rates[k]
    #fx_rates[v].diff() = fx_rates[v] - fx_rates[v].shift(1)
    strat = fx_rates[v].diff()
    reg = regression_based_performance(factor,strat,0)
    beta_currency = reg[0][0]
    treynor_ratio = reg[1]
    information_ratio = reg[2]
    alpha = reg[3]
    r_squared = reg[4]
    fx_hldg_reg.append(pd.DataFrame([[alpha,beta_currency,r_squared]],columns=['Alpha','Beta','R-Squared'],index = [k[4:7]]))


fx_hldg_reg_summary = pd.concat(fx_hldg_reg)
fx_hldg_reg_summary = fx_hldg_reg_summary.T
fx_hldg_reg_summary

In [ ]:
y = gmo_total[['SPY']]
X = signals[['DP', 'EP','US10Y']].shift(1)
reg = regression_based_performance(X,y)
beta_dp = reg [0][0]
beta_ep = reg[0][1]
beta_rf = reg[0][2]
alpha = reg[3]
r_squared = reg[4]
forecast = pd.DataFrame([[beta_dp,beta_ep,beta_rf,alpha,r_squared]],columns=['DP Beta','EP Beta','RF Beta','Alpha','R-Squared'],index = ['3_factors'])
forecast

# Time-series Regression

In [ ]:
#only one factor time series regression
def time_series_regression(portfolio, factors, FF3F = False, resid = False, scale =12):
    '''
    Input portfolio (columns of funds to be regressed), and factors.
    This function returns the portfolio columns as index, and regression estimates as columns
    If factors contain Fama-French 3 Factors, (Size and Value), then includes those betas.
    If resid = True, then we return the residual of each column in portfolio indexed by time
    '''
    
    report = pd.DataFrame(index = portfolio.columns)
    residual = pd.DataFrame(columns= portfolio.columns)

    for col in portfolio.columns:
        fund_ret = portfolio[col]
        model = sm.OLS(fund_ret, sm.add_constant(factors), missing= 'drop').fit()
        report.loc[col, 'Alpha'] = model.params['const'] * scale
        report.loc[col, 'Market Beta'] = model.params[1]
        if FF3F:
            report.loc[col, 'Size Beta'] = model.params[2]
            report.loc[col, 'Value Beta'] = model.params[3]
        report.loc[col, 'Information Ratio'] = np.sqrt(scale) * model.params['const'] / model.resid.std()
        report.loc[col, 'Treynor Ratio'] = scale * portfolio[col].mean() / model.params[1]
        report.loc[col, 'R-Squared'] = model.rsquared
        report.loc[col, 'Tracking Error'] = (model.resid.std()*np.sqrt(scale))  
        if resid:
            residual[col] = model.resid
    if resid:
        return residual
    return report

In [ ]:
#multiple factor time series regression
def time_series_regression(portfolio, factors, resid = False, scale =12):
    '''
    Input portfolio (columns of funds to be regressed), and factors.
    This function returns the portfolio columns as index, and regression estimates as columns
    Function includes betas for anything included in factors
    If resid = True, then we return the residual of each column in portfolio indexed by time
    '''
    
    report = pd.DataFrame(index = portfolio.columns)
    residual = pd.DataFrame(columns= portfolio.columns)

    for col in portfolio.columns:
        fund_ret = portfolio[col]
        model = sm.OLS(fund_ret, sm.add_constant(factors), missing= 'drop').fit()
        report.loc[col, 'Mean Return'] = scale * fund_ret.mean()
        report.loc[col, 'Sharpe Ratio'] = np.sqrt(scale) * fund_ret.mean() / fund_ret.std()
        report.loc[col, 'Alpha'] = model.params['const'] * scale
        for i, factor in enumerate(factors.columns):
            report.loc[col, f'{factor} Beta'] = model.params[i + 1]
        report.loc[col, 'Information Ratio'] = np.sqrt(scale) * model.params['const'] / model.resid.std()
        report.loc[col, 'Treynor Ratio'] = scale * portfolio[col].mean() / model.params[1]
        report.loc[col, 'R-Squared'] = model.rsquared
        report.loc[col, 'Tracking Error'] = (model.resid.std()*np.sqrt(scale))  
        if resid:
            residual[col] = model.resid
    if resid:
        return residual
    return report

In [ ]:
#multiple sample
reg_sub_sample = []
for k,v in sub_samples.items():    
    y = gmo.loc[sub_samples[k][0]:sub_samples[k][1],['GMWAX']].dropna()
    X = gmo.loc[y.index[0]:y.index[-1],['SPY']]
    reg = regression_based_performance(X,y)
    beta_mkt = reg[0][0]
    alpha = reg[3]
    r_squared = reg[4]
    reg_sub_sample.append(pd.DataFrame([[beta_mkt,alpha,r_squared]],columns=['SPY Beta','Alpha','R-Squared'],index = [k]))

reg_performance = pd.concat(reg_sub_sample)
reg_performance

In [ ]:
# one factor
factor = spy_er_ltcm['SPY']
reg_sum = []
for rets in ['Gross Excess Returns','Net Excess Returns']:
    fund_ret = ltcm_returns.loc[:, [rets]]
    reg = regression_based_performance(factor, fund_ret, 0)
    beta_mkt = reg[0][0]
    treynor_ratio = reg[1][0]
    information_ratio = reg[2]
    alpha = reg[3]
    r_squared = reg[4]
    reg_sum.append(pd.DataFrame([[alpha, beta_mkt, r_squared, treynor_ratio, information_ratio]], columns = ['Alpha', 'Market Beta','R-Squared','Treynor Ratio','Information Ratio'], index=[rets]))

mkt_reg_sum = pd.concat(reg_sum)
mkt_reg_sum

In [ ]:
# two factors
spy_er_ltcm['$SPY^{2}$'] = spy_er_ltcm['SPY']**2
factor_squared = spy_er_ltcm.loc[:, ['SPY', '$SPY^{2}$']]
quad_reg_sum = []
for rets in ['Gross Excess Returns','Net Excess Returns']:
    fund_ret = ltcm_returns.loc[:, [rets]]
    reg_squared = regression_based_performance(factor_squared, fund_ret, 0)
    #这里需要自己改：有几个beta就写几个beta
    beta_mkt = reg_squared[0][0]
    beta_mkt_squared = reg_squared[0][1]
    #------------------------------------------
    treynor_ratio = reg_squared[1][0]
    information_ratio = reg_squared[2]
    alpha = reg_squared[3]
    r_squared = reg_squared[4]
    #这里需要改beta的名称
    quad_reg_sum.append(pd.DataFrame([[alpha, beta_mkt, beta_mkt_squared, r_squared, treynor_ratio, information_ratio]], columns = ['Alpha', 'Market Beta', '$SPY^{2}$ Beta', 'R-Squared','Treynor Ratio','Information Ratio'], index=[rets]))
    #------------------------------------------

mkt_quad_reg_sum = pd.concat(quad_reg_sum)
mkt_quad_reg_sum

# Cross-sectional Regression

In [ ]:
#cross sectional test:
portfolio = portfolio_ret.mean().to_frame('Mean Portfolio excess returns')
cs_CAPM = time_series_regression(portfolio, ts_CAPM['Market Beta'])

In [ ]:
#tangent weight
def tangency_portfolio_rfr(asset_return,cov_matrix):
    """ 
        Returns the tangency portfolio weights in a (1 x n) vector when a riskless assset is available
        Inputs: 
            asset_return - Excess return over the risk free rate for each asset (n x 1) Vector
            cov_matrix = nxn covariance matrix for the assets
    """
    asset_cov = np.array(cov_matrix)
    inverted_cov= np.linalg.inv(asset_cov)
    one_vector = np.ones(len(cov_matrix.index))
    
    den = (one_vector @ inverted_cov) @ (asset_return)
    num =  inverted_cov @ asset_return
    return (1/den) * num

In [ ]:
#Tangency portfolio instruction
mean_ret = factor_summary['Mean']
tan_weights = tangency_portfolio_rfr(mean_ret, factors.cov())
tan_weights = pd.DataFrame(tan_weights, index = factors.columns, columns = ['Tangent Weights'])

#put mean and tangent weights together 
tan_mean_weights = pd.concat([tan_weights.T, factor_summary[['Mean']].T]).T
tan_mean_weights

# Mean Absolute Error (MAE)

In [ ]:
pd.DataFrame([(((ts_CAPM['alpha_hat']).abs().mean()))], columns = ['MAE (%)'], index = ['CAPM'])

In [ ]:
summary.loc[:, summary.columns.str.contains('Alpha')].abs().mean().to_frame('MAE')

In [ ]:
def calculate_mae(regression_result, scale = 12):
    return round(abs(regression_result['Alpha']).mean(), 6)

#Mean Absolute Value Instruction
mae_df = pd.DataFrame({
    'Model': ['CAPM', 'FF3', 'FF5', 'AQR'],
    'MAE': [calculate_mae(CAPM_regression), calculate_mae(FF3_regression), calculate_mae(FF5_regression), calculate_mae(AQR_regression)]
})

In [ ]:
def process_cs_regression(mean_data, beta_data, model_name):
    def perform_cs_regression(mean_data, beta_data, model_name):
        CS_result = sm.OLS(mean_data, beta_data, missing='drop').fit()
        CS_MAE = abs(CS_result.resid).mean()
        CS_premia = CS_result.params.to_frame(f'{model_name} Premia')
        return CS_result, CS_MAE, CS_premia
    CS_result, CS_MAE, premia = perform_cs_regression(mean_data, beta_data, model_name)
    premia.rename(index=lambda x: x.replace(f' {model_name} Beta', f' {model_name}'), inplace=True)
    return CS_result, CS_MAE, premia

In [ ]:
#一定要记得乘以12
#如果直接用time_series regression就不需要
portfolio = portfolios_ex.mean()*12.to_frame('Mean Portfolio excess returns')
# Assuming 'factor_returns' is the DataFrame that contains the factor returns for MKT, HML, and UMD.
cs_CAPM = time_series_regression(portfolio, ts_regression[['MKT Beta', 'HML Beta', 'UMD Beta']])
cs_CAPM = cs_CAPM[['Alpha', 'MKT Beta Beta','HML Beta Beta', 'UMD Beta Beta', 'R-Squared' ]]
cs_CAPM.rename(columns={'Alpha': 'annualized intercept', 'MKT Beta Beta': 'annualized market beta', 'HML Beta Beta': 'annualized value beta','UMD Beta Beta': 'annualized momentum beta','R-Squared':'R-Squared'}, inplace=True)
cs_CAPM

In [ ]:
#premia 就是beta的值（就是mean是左边，右边是之前算出来的几个factor的beta值，用beta的值作为regressor来算beta --> premia
#用在time series里面每个factor的mean和在cross sectional test里面的premia（beta的值）进行比较，接近说明好
AQR_CS, AQR_CS_MAE, AQR_CS_premia = process_data_regression(AQR_mean, AQR_beta, 'AQR')
FF3_CS, FF3_CS_MAE, FF3_CS_premia = process_data_regression(FF3_mean, FF3_beta, 'FF3')
FF5_CS, FF5_CS_MAE, FF5_CS_premia = process_data_regression(FF5_mean, FF5_beta, 'FF5')

cs_premia = pd.concat([mean_ex_ret, AQR_CS_premia, FF3_CS_premia, FF5_CS_premia], axis = 1).fillna('')
display(cs_premia)


cs_mae = pd.Series([AQR_CS_MAE, FF3_CS_MAE, FF5_CS_MAE], index = ['AQR', 'FF3', 'FF5'])
mae_ts = mae_df.loc[['AQR', 'FF3', 'FF5']]
mae = pd.concat([mae_ts, cs_mae], axis = 1)
mae.columns = ['Time Series MAE', 'Cross Section MAE']
display(mae)

# OOS Forecast

In [ ]:
# function 

def OOS_r2(df, X, window):
    y = df['SPY']
    X = sm.add_constant(X)

    forecast_err, null_err = [], []

    # eumerate: for index,value in eumerate(iterable)
    for i,j in enumerate(df.index):
        if i >= window:
            # for an expanding window
            currX = X.iloc[:i]
            currY = y.iloc[:i]

            # apply OLS on X and Y
            reg = sm.OLS(currY, currX, missing = 'drop').fit()

            null_forecast = currY.mean()

            # regression prediction y value using x value
            reg_predict = reg.predict(X.iloc[[i]])

            actual = y.iloc[[i]]

            forecast_err.append(reg_predict - actual)
            null_err.append(null_forecast - actual)
            
    RSS = (np.array(forecast_err)**2).sum()
    TSS = (np.array(null_err)**2).sum()
    
    return ((1 - RSS/TSS),reg)

In [ ]:
def OOS_forecasting(df, factors, start):
    y = df
    X = sm.add_constant(factors)

    forecast_err, null_err,oos_predictions,null_predictions = [], [],[],[]

    for i,j in enumerate(df.index):
        if i >= start:
            currX = X.iloc[:i]
            currY = y.iloc[:i]
            reg = sm.OLS(currY, currX, missing = 'drop').fit()
            null_forecast = currY.mean()
            reg_predict = reg.predict(X.iloc[[i]])
            actual = y.iloc[[i]]
            oos_predictions.append(reg_predict.T)
            null_predictions.append(pd.DataFrame([[reg_predict.index[0]]], columns = ['Date'], index = [null_forecast]))
            forecast_err.append(reg_predict.values - actual)
            null_err.append(null_forecast.values - actual)
            
    RSS = (np.array(forecast_err)**2).sum()
    TSS = (np.array(null_err)**2).sum()
    predictions_df = pd.concat(oos_predictions).T.drop_duplicates()
    null_predictions_df = pd.concat(null_predictions).T
    
    return ((1 - RSS/TSS),reg,predictions_df,null_predictions_df)

In [ ]:
factor = signals.loc[:,'EP'].shift(1).to_frame()
fund_ret = gmo_total.loc[factor.index[0]:,['SPY']]
reg_ep = OOS_r2(fund_ret,factor,60)
OOS_RSquared_ep = reg_ep[0]
OOS_r2_ep = pd.DataFrame([[OOS_RSquared_ep]], columns = ['OOS R-Squared'], index = ['EP'])
reg_ep_params = reg_ep[1]

# OOS Dynamic Forecast Strategy

In [ ]:
# function

def OOS_strat(df, factors, start, weight):
    returns = []
    y = df['SPY']
    X = sm.add_constant(factors)

    for i,j in enumerate(df.index):
        if i >= start:
            # expanding window
            currX = X.iloc[:i]
            currY = y.iloc[:i]
            reg = sm.OLS(currY, currX, missing = 'drop').fit()
            pred = reg.predict(X.iloc[[i]])
            w = pred * weight
            returns.append((df.iloc[i]['SPY'] * w)[0])

    df_strat = pd.DataFrame(data = returns, index = df.iloc[-(len(returns)):].index, columns = ['Strat Returns'])
    return df_strat

In [ ]:
#create return rt+1 for SPY
factor = signals.loc[:,'EP'].shift(1).to_frame()
fund_ret= gmo_total.loc[factor.index[0]:,['SPY']]
OOS_EP_predict = OOS_strat(fund_ret,factor, 60, 100).rename(columns={'Strat Returns':'EP_OOS_Returns'})

In [ ]:
#concat
oos_prediction_sum = pd.concat([OOS_DP_predict.T,OOS_EP_predict.T,OOS_EPDP_predict.T,OOS_all_predict.T])
oos_prediction_sum = oos_prediction_sum.T
oos_prediction_sum

In [ ]:
#regress: fund_ret = rt+1(the prediction you calculated), factor = rt 
oos_prediction_sum = pd.concat([OOS_DP_predict.T,OOS_EP_predict.T,OOS_all_predict.T])
oos_prediction_sum = oos_prediction_sum.T

strats = {'DP': OOS_DP_predict.dropna(),
          'EP': OOS_EP_predict.dropna(),
          'DP-EP':OOS_EPDP_predict.dropna(),
          'All': OOS_all_predict.dropna(),
          'SPY':gmo_excess_ret.loc[OOS_all_predict.index[0]:,['SPY']].rename(columns={'SPY':'SPY_OOS_Returns'}),
          'US3M':rf['US3M'].to_frame('US3M_OOS_Returns')
         }
factor = gmo_excess_ret.loc[:,['SPY']]
strat_summary =[]
for k,v in strats.items():
    strat = strats[k]
    perf_summary = performance_summary(strat)
    perf_summary['Negative Risk Premium Months'] = len(strat[strat[k+'_OOS_Returns'] - rf['US3M'] <0])
    perf_summary['Total Months'] = len(strat)
    perf_summary.index = [k]
    reg = time_series_regression(strat, factor[strat.index[0]:].squeeze(), False)
    perf_summary['Market Beta'] = reg['SPY beta'].values
    perf_summary['Market Alpha'] = reg['alpha_hat'].values
    perf_summary['Market Information Ratio'] = reg['info_ratio'].values
    strat_summary.append(perf_summary)
    

strat_summary_df = pd.concat(strat_summary)
strat_summary_df.loc[:,['Mean','Volatility','Sharpe Ratio','VaR (0.05)','Max Drawdown','Market Beta','Market Alpha','Market Information Ratio']]

In [ ]:
#带年份：
oos_prediction_sum = pd.concat([OOS_DP_predict.T,OOS_EP_predict.T,OOS_all_predict.T])
oos_prediction_sum = oos_prediction_sum.T

strats = {'DP': OOS_DP_predict.dropna(),
          'EP': OOS_EP_predict.dropna(),
          'DP-EP':OOS_EPDP_predict.dropna(),
          'All': OOS_all_predict.dropna(),
          'US3M':rf['US3M'].to_frame('US3M_OOS_Returns')
         }
factor = gmo_excess_ret.loc[:,['SPY']]['2000':'2011']
strat_summary =[]
for k,v in strats.items():
    strat = strats[k]['2000':'2011']
    perf_summary = performance_summary(strat)
    perf_summary['Negative Risk Premium Months'] = len(strat[strat[k+'_OOS_Returns'] - rf['2000':'2011']['US3M'] <0])
    perf_summary['Total Months'] = len(strat)
    perf_summary.index = [k]
    reg = time_series_regression(strat, factor[strat.index[0]:].squeeze(), False)
    perf_summary['Market Beta'] = reg['SPY beta'].values
    perf_summary['Market Alpha'] = reg['alpha_hat'].values
    perf_summary['Market Information Ratio'] = reg['info_ratio'].values
    strat_summary.append(perf_summary)
    

strat_summary_df_0011 = pd.concat(strat_summary)
strat_summary_df_0011.loc[:,['Mean','Volatility','Sharpe Ratio','VaR (0.05)','Max Drawdown','Market Beta','Market Alpha','Market Information Ratio']]

# Rolling & Expanding

In [ ]:
expending_window = np.sqrt((rets**2).expanding(60).mean().shift())
rolling_window = np.sqrt((rets**2).rolling(60).mean().shift())

# Rolling OOS

In [ ]:
def rolling_regression_param(factor,fund_ret,roll_window = 60):
    """ 
        Returns the Rolling Regression parameters for given set of returns and factors
        Inputs:
            factor - Dataframe containing monthly returns of the regressors
            fund_ret - Dataframe containing monthly excess returns of the regressand fund
            roll_window = rolling window for regression
        Output:
            params - Dataframe with time-t as the index and constant and Betas as columns
    """
    X = sm.add_constant(factor)
    y= fund_ret
    rols = RollingOLS(y, X, window=roll_window)
    rres = rols.fit()
    params = rres.params.copy()
    params.index = np.arange(1, params.shape[0] + 1)
    return params

In [ ]:
rolling_model = RollingOLS(hfri, sm.add_constant(merrill), window=60).fit()  #rolling 
rolling_params = rolling_model.params
factor_ret = merrill.loc['2016-08-31':]    #60 month 说明从5年后开始
one_df = pd.DataFrame(np.ones(factor_ret.shape[0])).T
one_df.columns = factor_ret.index
factor_ret = pd.concat([one_df, pd.DataFrame(factor_ret).loc['2016-08-31':].T])  #第一行都是1

rolling_reg_replication = pd.DataFrame(np.diag(pd.DataFrame(np.array(
    rolling_params[60:]) @ np.array(factor_ret))), index=factor_ret.columns, columns=['HFRIFWI Index'])
performance_summary(rolling_reg_replication)

# Forecast/ Expected value/ Fitted Mean based on regression

In [ ]:
with_const = np.array(merrill) @ np.array(coefficients.loc[merrill.columns])
without_const = np.array(
    merrill) @ np.array(coefficients_new.loc[merrill.columns])
no_constant_replication = without_const.mean() * 12
original = hedge_fund['HFRIFWI Index'].mean() * 12
pd.DataFrame([[no_constant_replication, original]], columns=[
             'Replication', 'Actual'], index=['Fitted & Actual Mean'])

In [ ]:
regr_int = sm.OLS(hfri, sm.add_constant(mer_factors), missing="drop").fit()
regr_no_int = sm.OLS(hfri, mer_factors, missing="drop").fit()

# Calculate the fitted mean
fitted_mean = (regr_int.params[0] + regr_int.params[1:] @ mer_factors.mean()) * 12
fitted_mean_no_int = (regr_no_int.params @ mer_factors.mean()) * 12

# Get hfri mean
hfri_mean = hfri.mean()  * 12

In [ ]:
dp_return = (dp_forecast['DP Beta'] * signals['DP'].shift(1).to_frame() + dp_forecast['Alpha']/12)*100

# subsample

In [ ]:
#two sample for momentum
BU = size_sorts_tot['BIG HiPRIOR']
SU = size_sorts_tot['SMALL HiPRIOR']
BD = size_sorts_tot['BIG LoPRIOR']
SD = size_sorts_tot['SMALL LoPRIOR']
RF = rf_rate['RF']
ports = (1/2 * (BU + SU) - RF).to_frame('LONG ONLY')
ports['LONG AND SHORT'] = 1/2 * ((BU + SU) - (BD + SD))
period = ('1994', '2023')
mom_perf = []

for port in ports.columns: 
    sub_mom = ports[[port]].loc[period[0]:period[1]]
    summary = performance_summary(sub_mom)
    sub_factors = factors_ex.loc[period[0]: period[1]]
    summary['Corr Mkt'] = np.corrcoef(np.array(sub_factors['MKT']), np.array(sub_mom[port]))[0,1]
    summary['Corr Val'] = np.corrcoef(np.array(sub_factors['HML']), np.array(sub_mom[port]))[0,1]
    summary['Corr Size'] = np.corrcoef(np.array(sub_factors['SMB']), np.array(sub_mom[port]))[0,1]
    mom_perf.append(summary)

mom_perf = pd.concat(mom_perf)
display(mom_perf[['Mean', 'Volatility', 'Sharpe Ratio', 'Skewness', 'Corr Mkt', 'Corr Val']].style.format('{:.4f}'))

In [ ]:
#log level
agg_years = [(1965, 1999), (2000, 2023), (1926, 2023)]
def calc_analytics_by_year(data, agg_years):
    res = []
    for y in agg_years:
        sub = data.loc[str(y[0]):str(y[1])]
        res.append({'mean': sub.mean() * 12, 'vol': sub.std() * np.sqrt(12)})
    return pd.DataFrame(res, index = [f'{i[0]} - {i[1]}' for i in agg_years]).stack()

sum_stats = data.apply(calc_analytics_by_year, agg_years = agg_years).T
log_sum_stats = np.log(1 + data).apply(calc_analytics_by_year, agg_years = agg_years).T

res_stats = pd.concat([sum_stats, log_sum_stats])
res_stats.index = pd.MultiIndex.from_product([['levels','logs'], sum_stats.index.to_list()])
res_stats.style.format('{:,.2%}')

In [ ]:
#log level
stats_table = pd.DataFrame(
    index=pd.MultiIndex.from_tuples([('levels', 'SPX'), ('levels', 'RiskFree'), (
        'levels', 'Excess SPX'), ('logs', 'SPX'), ('logs', 'RiskFree'), ('logs', 'Excess SPX')]),
    columns=pd.MultiIndex.from_tuples([('1965-1999', 'mean'), ('1965-1999', 'vol'), ('2000-2023', 'mean'), ('2000-2023', 'vol'), ('1926-2023', 'mean'), ('1926-2023', 'vol')]))


stats_table[('1965-1999', 'mean')] = ((df.loc['1965':'1999']).mean()* 12).tolist() + ((np.log(1+df.loc['1965':'1999'])).mean()*12).tolist()
stats_table[('1965-1999', 'vol')] = ((df.loc['1965':'1999']).std() * np.sqrt(12)).tolist() + ((np.log(1+df.loc['1965':'1999'])).std() * np.sqrt(12)).tolist()
stats_table[('2000-2023', 'mean')] = ((df.loc['2000':'2023']).mean()* 12).tolist() + ((np.log(1+df.loc['2000':'2023'])).mean()*12).tolist()
stats_table[('2000-2023', 'vol')] = ((df.loc['2000':'2023']).std() * np.sqrt(12)).tolist() + ((np.log(1+df.loc['2000':'2023'])).std() * np.sqrt(12)).tolist()
stats_table[('1926-2023', 'mean')] = (np.mean(df) * 12).tolist() + (np.mean(np.log(1+df))*12).tolist()
stats_table[('1926-2023', 'vol')] = (df.std()* np.sqrt(12)).tolist() + ((np.log(1+df)).std() * np.sqrt(12)).tolist()

display(stats_table.style.format('{:,.2%}'))

In [ ]:
#different time 
sub_1980 = factors.loc[:'1980']
sub_2001 = factors.loc['1981':'2001']
sub_2022 = factors.loc['2002':]

df_dict={'1926-1980' : sub_1980,
         '1981-2001' : sub_2001,
         '2002-2022' : sub_2022}

summary_lst = []
for key in df_dict.keys():
    summary_stats = performance_summary(df_dict[key]).loc[:,['Mean','Volatility','Sharpe Ratio','VaR (0.05)']]
    summary_stats['Period'] = key
    summary_stats= summary_stats.reset_index().rename(columns = {'index':'Factor'}).set_index(['Period','Factor'])
    summary_lst.append(summary_stats)

factor_summary = pd.concat(summary_lst)
factor_summary



#another version
sub_samples = {
              '1993-2011' : ['1993','2011'],
              '2012-2023' : ['2012','2023'],
              '1993-2023' : ['1993','2023'],
              }

gmo_sum = []
for k,v in sub_samples.items():
    sub_gmo = gmo.loc[sub_samples[k][0]:sub_samples[k][1],['GMWAX']].dropna()
    gmo_summary = performance_summary(sub_gmo)
    gmo_summary = gmo_summary
    gmo_summary.index = [k]
    gmo_sum.append(gmo_summary)

gmo_summary = pd.concat(gmo_sum)
gmo_summary.loc[:,['Mean','Volatility','Sharpe Ratio']]

In [ ]:
# momentum different time
periods = [['1927', '2022'], ['1927', '1993'], ['1994', '2008'], ['2009', '2023']]
summary_col_names = ['Annualized Return','Annualized Volatility','Annualized Sharpe Ratio','Skewness']
res = []
for period in periods:
    temp = momentum.loc[period[0]:period[1]]
    temp_ff = ff_factors.loc[period[0]:period[1]]
    summary = calc_performance_metrics(temp)[summary_col_names]
    summary['mkt_corr'] = temp_ff.corr().loc['MKT',['UMD']]
    summary['val_corr'] = temp_ff.corr().loc['HML',['UMD']]
    summary = summary.T.iloc[:,0].rename(f'{period[0]} - {period[1]}')
    res.append(summary)
summary  = pd.concat(res, axis=1).T
summary

# Probability

In [ ]:
def prob(mu, sigma, h):
    return norm.cdf(np.sqrt(h)*mu/sigma)

In [ ]:
strike = np.log(1.06)
mean_return = np.log(1+data['SPX'].loc['1965':]).mean() * 12
sigma = np.log(1 + data['SPX'].loc['1965':]).std() * np.sqrt(12)
performance = np.log(1+data['SPX'].loc['2000':]).mean() * 12
mu = 24/6 * (strike - performance) + strike - mean_return

print(f'Probability of Puts ending in the money: {prob(mu, sigma, 6):,.2%}')

# CVAR VAR

In [ ]:
#iid, cdf, norm
VaR:  = -1.65 * std
cVaR: = 0-norm.pdf(1.65) / 0.05 * std

In [ ]:
exceed_historical = forecasting[forecasting['GLD'] < gld_performance['VaR (0.05)'].values[0]]['GLD'].count()*100/len(forecasting['GLD'])
exceed_norm_full = forecasting[forecasting['GLD'] < VaR_full_sample.values[0]]['GLD'].count()*100/len(forecasting['GLD'])
exceed_norm_roll = sum(forecasting['GLD'].loc[rolling_var.index] < rolling_var['Rolling VaR'])*100/len(rolling_var['Rolling VaR'])
pd.DataFrame([[exceed_historical,exceed_norm_full,exceed_norm_roll]], columns = ['Historical','Normal (Full Sample)','Normal (Rolling)'], index= ['% Data below 5% VaR backtest'])

# Heatmap

In [ ]:
factor_corr = factors.corr()

plt.figure(figsize=(16, 6))
heatmap = sns.heatmap(factor_corr, vmin=0, vmax=1, annot=True)
heatmap.set_title('Factors Correlation Heatmap', fontdict={'fontsize':12}, pad=12)

In [ ]:
#three heatmap together 
fig, (ax,ax2,ax3) = plt.subplots(ncols=3)

# fig.subplots_adjust(wspace=0.01)
# fig.tight_layout(pad=1)
sns.heatmap(sub_1980.corr(), ax=ax, cbar=False, annot = True).set_title('1926 - 1980', fontdict={'fontsize':12}, pad=12)

fig.colorbar(ax.collections[0], ax=ax,location="left", use_gridspec=False, pad=0.2)
sns.heatmap(sub_2001.corr(), ax=ax2, cbar=False, annot = True).set_title('1981 - 2001', fontdict={'fontsize':12}, pad=12)

fig.colorbar(ax.collections[0],ax=ax2,location="right", use_gridspec=False, pad=0.2)

sns.heatmap(sub_2023.corr(), ax=ax3, cbar=False, annot = True).set_title('2002 - 2023', fontdict={'fontsize':12}, pad=12)
fig.colorbar(ax.collections[0],ax=ax3,location="right", use_gridspec=False, pad=0.2)

fig.set_figwidth(15)
plt.show()

In [ ]:
# Plot cumulative returns of factors, add vertical line at 2015
fig, ax = plt.subplots(figsize=(8, 5))
cum_rets = (1 + factors).cumprod() - 1
cum_rets.plot(ax=ax)
ax.axvline('2015', color='k', linestyle='--')
ax.set_title('Cumulative Returns of Factors')
ax.set_ylabel('Cumulative Return')
ax.set_xlabel('Date');



plt.plot((1 + ports['1994':]).cumprod(), label = ports.columns)
plt.title('Cumulative Return since 1994')
plt.legend()
plt.show()

# Figure

In [ ]:
#cumulative return figures
figure = ((factors + 1).cumprod()).plot()
# plt.figure(figsize=(100, 6))
plt.title('Cumulative Returns')
plt.show()

In [ ]:
#two cumulative return figures
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.suptitle('Sub Period Cumulative Returns')
ax1.plot(((sub_2001 + 1).cumprod()*100)-100)
ax2.plot(((sub_2023 + 1).cumprod()*100)-100)

fig.set_figwidth(15)
ax1.legend(sub_2001.columns)
ax2.legend(sub_2023.columns)

ax1.title.set_text('1981-2001')
ax2.title.set_text('2002-2023')

In [ ]:
#scatterplot
plt.scatter(portfolio_summary.loc[:,['Volatility']],portfolio_summary.loc[:,['Mean']])
plt.xlabel("Volatility")
plt.ylabel("Mean Excess Returns")

## Dynamic Carry Trade
Use this to write $\mathbb{E}_{t}[\tilde{r}^{i}_{t+1}]$ as a function of the interest-rate differential as well as $\alpha$ and $\beta$ from this FX regression.<br>


\begin{align}
\mathbb{E}_{t}[{s}_{t+1} - {s}_{t}] = \alpha + \beta({r}^{f,\$}_{t,t+1} -  {r}^{f,i}_{t,t+1})
\end{align}

Then use the definition of excess (log) returns on FX:<br>

\begin{align}
\tilde{r}^{i}_{t+1} = {s}_{t+1} - {s}_{t} - ({r}^{f,\$}_{t,t+1} -  {r}^{f,i}_{t,t+1})
\end{align}


Rearranging, this implies the following forecast for excess log returns:<br>

\begin{align}
\mathbb{E}_[\tilde{r}^{i}_{t+1}] = \alpha + (\beta-1) ({r}^{f,\$}_{t,t+1} -  {r}^{f,i}_{t,t+1})
\end{align}

### 3.4.a) Use your regression estimates from Problem 3 along with the formula above to calculate the fraction of months for which the estimated FX risk premium positive. That is, for each i, calculate how often in the time-series we have <br>

\begin{align}
\mathbb{E}_{t}[\tilde{r}^{i}_{t+1}] > 0
\end{align}

# Positive Premium

In [ ]:
fx_prem_lst = []
for k,v in fx_spot_map.items():
    fx_er_usd = (risk_free_rates['log_USD1M'].shift(1) - risk_free_rates[k].shift(1)).to_frame('ER_over_USD')
    expected_fx_premium = float(fx_regression_results.loc[[k[4:7]], 'Alpha'])/12 + (fx_er_usd.loc[:,['ER_over_USD']]  * float(fx_regression_results.loc[[k[4:7]], 'Beta'] - 1))
    expected_fx_premium = expected_fx_premium.rename(columns={'ER_over_USD':k[4:7]})
    positive_premium =  len(expected_fx_premium[expected_fx_premium[k[4:7]] > 0])
    fx_prem_lst.append(pd.DataFrame([[positive_premium,len(expected_fx_premium),positive_premium*100/len(expected_fx_premium)]],columns=['Months - Positive Premium','Total Months','Frequency(%)-Positive Premium'],index=[k[4:7]]))
fx_premium = pd.concat(fx_prem_lst)
fx_premium




# Negative premium month

In [ ]:
strats = {'DP': dp_return.dropna(),
          'EP': ep_return.dropna(),
          'DP-EP-US10Y': factors_return.dropna()
         }
factor = gmo.loc[:,['SPY']]
strat_summary =[]
for k,v in strats.items():
    strat = strats[k]
    perf_summary = performance_summary(strat)
    perf_summary['Negative Risk Premium Months'] = len(strat[strat['Forecasted Return'] - rf['US3M'] <0])
    perf_summary['Total Months'] = len(strat)
    perf_summary.index = [k]
    # X is SPY, y is returns predicted by signals
    reg = regression_based_performance(factor[strat.index[0]:],strat)
    perf_summary['Market Beta'] = reg[0][0]
    perf_summary['Market Alpha'] = reg[3]
    perf_summary['Market Information Ratio'] = reg[2]
    strat_summary.append(perf_summary)
    

strat_summary_df = pd.concat(strat_summary)
strat_summary_df.loc[:,['Mean','Volatility','Sharpe Ratio','Max Drawdown','Market Beta','Market Alpha','Market Information Ratio']]